<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

# 2.3 Data Access and Basic Processing
<img src="https://raw.githubusercontent.com/pangeo-data/pangeo.io/refs/heads/main/public/Pangeo-assets/pangeo_logo.png"
     alt="Pangeo logo"
     style="float: center; margin-right: 10px; max-height: 80px;"/>
     
## Reduce Operators with Pangeo ecosystem

When computing statistics over time or indices based on multiple bands, it is possible to use reduce operators.

In Pangeo and `Xarray` we can use different methods for reducing the dimensions, such as `median`, `mean` or `groupby`, which applies a reducer to a data cube dimension by collapsing all the values along the specified dimension into an output value computed by the reducer.

Reduce the temporal dimension to a single value, the mean for instance:

In [ ]:
import pystac_client
import stackstac
import xarray as xr
import numpy as np

In [ ]:
#                  West,     South,     East,      North
spatial_extent = [11.259613, 46.461019, 11.406212, 46.522237]
temporal_extent = ["2021-05-28T00:00:00Z","2021-06-30T00:00:00Z"]

In [ ]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
s2_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["sentinel-2-l2a"]
).item_collection()

s2_cube = stackstac.stack(s2_items,
                     bounds_latlon=spatial_extent,
                     assets=["red","nir"]
)
s2_cube

### Get rid of possible negative values

In [ ]:
s2_cube = xr.where(s2_cube>=0, s2_cube, 0)
s2_cube

Reduce the `time` dimension by averaging along the `time` dimension.

We can use the `Xarray` [reduce](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.reduce.html) method, passing the reducer function and the dimension to be reduced:

In [ ]:
datacube_mean_time = s2_cube.reduce(np.mean,dim="time")

Alternatively, `Xarray` define convenience methods, allowing to call the reduce operation over time easily with the following syntax, producing the same result. Check what happens to the datacube inspecting the resulting Xarray object.

In [ ]:
datacube_mean_time = s2_cube.mean("time")
datacube_mean_time

It is possible to reduce in the same way all the available dimensions of the datacube.

We can, for instance, reduce the band dimension similarly as we did for the temporal dimension:

The result will now contain values resulting from the average of the bands:

In [ ]:
datacube_mean_band = s2_cube.mean("band")
datacube_mean_band

**Quiz hint: look carefully at number of pixels of the loaded datacube!**

The reducer could be again a single process, but when computing spectral indices like NDVI, NDSI etc. an arithmentical formula is used instead.

For instance, the [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) formula can be expressed using a selection (`sel`) method over the `band` dimension:

$$ NDVI = {{NIR - RED} \over {NIR + RED}} $$

In [ ]:
def NDVI(data):
    red = data.sel(band="red")
    nir = data.sel(band="nir")
    ndvi = (nir - red)/(nir + red)
    return ndvi

ndvi_xr = NDVI(s2_cube)
ndvi_xr

Visualize a sample NDVI result:

In [ ]:
%%time
ndvi_xr.isel(time=0).plot.imshow(vmin=-1,vmax=1,cmap="Greens")

Additionally, it is possible to reduce both spatial dimensions of the datacube at the same time.

To do this, we need to reduce the spatial dimension using for instance `median`.

This time we select a smaller area of interest, to reduce the amount of downloaded data:

In [ ]:
#                  West,     South,     East,      North
spatial_extent = [11.31369, 46.52167, 11.31906, 46.52425]
temporal_extent = ["2021-01-01T00:00:00Z","2021-12-30T00:00:00Z"]
bands = ["red","nir"]

In [ ]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
s2_items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["sentinel-2-l2a"]
).item_collection()

s2_cube = stackstac.stack(s2_items,
                     bounds_latlon=spatial_extent,
                     assets=bands
)
s2_cube = s2_cube[s2_cube["eo:cloud_cover"] < 35]

### Get rid of possible negative values

In [ ]:
s2_cube = xr.where(s2_cube>=0, s2_cube, 0)

### Reduce dimension x and y with median values

In [ ]:
datacube_spatial_median = s2_cube.median(dim=["x", "y"])

Verify that the spatial dimensions were collapsed:

In [ ]:
datacube_spatial_median

We can combine this spatial reducer with the previous over bands to compute a time series of NDVI values:

In [ ]:
ndvi_spatial_xr = NDVI(datacube_spatial_median)
ndvi_spatial_xr

In [ ]:
ndvi_spatial_xr = ndvi_spatial_xr.compute()
ndvi_spatial_xr

**Quiz hint: look carefully at the dimensions of the resulting datacube!**

`compute()` executes the Dask computation and blocks until the result is available. It then collects and returns the final result to the local process.
In this case it will trigger the download of the data from the STAC Catalog and the computing the NDVI time series.

Visualize the NDVI time series:

In [ ]:
ndvi_spatial_xr.where(ndvi_spatial_xr<1).plot.scatter()